In [54]:
from functions_v2 import*
from methods import MethodFinder

import brightway2 as bw
import bw2data as bd
import bw2analyzer as ba

#reduce?
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [55]:
bd.projects.set_current("premise-validation-try1")
bw.bw2setup()

bio3=bw.Database('biosphere3')
ei39=bw.Database('ecoinvent 3.9.1 cutoff')
ei39SSP2=bw.Database('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27')

Biosphere database already present!!! No setup is needed


In [56]:
#Get 5 sectors from 5 yaml files and run LCA scores to plot lvl 1 plot. Save each method in a different sheet.

LCA DATA TO EXCEL WORKFLOW

In [67]:
#sector filters file names/paths

cement = 'cement_small.yaml'
electricity = 'electricity_small.yaml'
fuels= 'fuels_small.yaml'
steel = 'steel_small.yaml'
transport = 'transport_small.yaml'

In [138]:
files_dict={}
files_dict['Cement']={'yaml': 'yamls\cement_small.yaml',
                      'yaml identifier': 'Cement'}
files_dict['Electricity']= {'yaml':'yamls\electricity_small.yaml',
                            'yaml identifier': 'Electricity'} #yaml identifier is the name of the filter in the yaml file, in the first line.
files_dict

{'Cement': {'yaml': 'yamls\\cement_small.yaml', 'yaml identifier': 'Cement'},
 'Electricity': {'yaml': 'yamls\\electricity_small.yaml',
  'yaml identifier': 'Electricity'}}

In [139]:

yaml_file=files_dict['Cement']['yaml']
cement_sector_activities=generate_sets_from_filters(yaml_file, database=ei39SSP2)
cement_sector_activities_to_list=list(cement_sector_activities['Cement'])[:]
cement_sector_activities_to_list
main_dict=files_dict.copy()
main_dict['Cement']['activities']=cement_sector_activities_to_list
main_dict


{'Cement': {'yaml': 'yamls\\cement_small.yaml',
  'yaml identifier': 'Cement',
  'activities': ['cement production, Portland' (kilogram, IN, None),
   'cement production, Portland' (kilogram, CH, None),
   'cement production, Portland' (kilogram, US, None),
   'cement production, Portland' (kilogram, ZA, None),
   'cement production, Portland' (kilogram, CA-QC, None),
   'cement production, Portland' (kilogram, BR, None),
   'cement production, Portland' (kilogram, PE, None)]},
 'Electricity': {'yaml': 'yamls\\electricity_small.yaml',
  'yaml identifier': 'Electricity'}}

In [140]:
yaml_file=files_dict['Electricity']['yaml']
electricity_sector_activities=generate_sets_from_filters(yaml_file, database=ei39SSP2)
electricity_sector_activities_to_list=list(electricity_sector_activities['Electricity'])[:]
electricity_sector_activities_to_list
main_dict=files_dict.copy()
main_dict['Electricity']['activities']=electricity_sector_activities_to_list
main_dict

{'Cement': {'yaml': 'yamls\\cement_small.yaml',
  'yaml identifier': 'Cement',
  'activities': ['cement production, Portland' (kilogram, IN, None),
   'cement production, Portland' (kilogram, CH, None),
   'cement production, Portland' (kilogram, US, None),
   'cement production, Portland' (kilogram, ZA, None),
   'cement production, Portland' (kilogram, CA-QC, None),
   'cement production, Portland' (kilogram, BR, None),
   'cement production, Portland' (kilogram, PE, None)]},
 'Electricity': {'yaml': 'yamls\\electricity_small.yaml',
  'yaml identifier': 'Electricity',
  'activities': ['electricity production, at biomass-fired IGCC power plant' (kilowatt hour, CEU, None),
   'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, BRA, None),
   'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, USA, None),
   'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, TUR, None),
   'electricity production, at biomass-fired 

In [142]:
# Assuming generate_sets_from_filters is a predefined function
def process_yaml_files(files_dict, database):
    main_dict=files_dict.copy()
    for key, value in main_dict.items():
        yaml_file = value['yaml']
        yaml_identifier = value['yaml identifier']
        
        # Generate the sector activities
        sector_activities = generate_sets_from_filters(yaml_file, database=database)
        
        # Convert the set of activities to a list
        activities_list = list(sector_activities[yaml_identifier])
        
        # Add to the sectors_dict
        main_dict[key]['activities'] = activities_list
        
    return main_dict


{'Cement': {'yaml': 'yamls\\cement_small.yaml', 'yaml identifier': 'Cement', 'activities': ['cement production, Portland' (kilogram, IN, None), 'cement production, Portland' (kilogram, US, None), 'cement production, Portland' (kilogram, CH, None), 'cement production, Portland' (kilogram, ZA, None), 'cement production, Portland' (kilogram, CA-QC, None), 'cement production, Portland' (kilogram, BR, None), 'cement production, Portland' (kilogram, PE, None)]}, 'Electricity': {'yaml': 'yamls\\electricity_small.yaml', 'yaml identifier': 'Electricity', 'activities': ['electricity production, at biomass-fired IGCC power plant' (kilowatt hour, CEU, None), 'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, BRA, None), 'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, USA, None), 'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, TUR, None), 'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, RUS, 

In [111]:
sectors_dict= process_yaml_files(files_dict, ei39SSP2)

In [122]:
sectors_dict['Electricity']

{'activities': ['electricity production, at biomass-fired IGCC power plant' (kilowatt hour, CEU, None),
  'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, BRA, None),
  'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, USA, None),
  'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, TUR, None),
  'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, RUS, None),
  'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, RSAM, None),
  'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, SAF, None),
  'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, CHN, None),
  'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, KOR, None),
  'electricity production, at biomass-fired IGCC power plant' (kilowatt hour, UKR, None)]}

In [144]:
#Get Methods
finder=MethodFinder()
finder.find_and_create_method(criteria=['IPCC', '2013', 'GWP100'], exclude=['no LT'])
finder.find_and_create_method(criteria=['EN15804','Cumulative', 'non-renewable' ])
# finder.find_and_create_method(criteria=['land occupation','selected'])
# finder.find_and_create_method(criteria=['EN15804','fresh water'])
method_dict=finder.get_all_methods()
method_dict

{'method_1': {'object': Brightway2 Method: IPCC 2013: climate change: global warming potential (GWP100),
  'method name': ('IPCC 2013',
   'climate change',
   'global warming potential (GWP100)'),
  'short name': 'global warming potential (GWP100)',
  'unit': 'kg CO2-Eq'},
 'method_2': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: Cumulative Energy Demand - non-renewable energy resources,
  'method name': ('EN15804',
   'inventory indicators ISO21930',
   'Cumulative Energy Demand - non-renewable energy resources'),
  'short name': 'Cumulative Energy Demand - non-renewable energy resources',
  'unit': 'megajoule'}}

In [145]:
sector_activities=main_dict['Cement']['activities']

lca_scores=compare_activities_multiple_methods(
        activities_list=sector_activities,
        methods=method_dict,
        identifier='Cement',
        mode='absolute'
    )

lca_scores=small_inputs_to_other_column(lca_scores, cutoff=0.02)

scores_dict=main_dict.copy()
scores_dict['Cement']['lca_tables_different_methods']=lca_scores

Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '


In [149]:
scores_dict['Cement']['lca_tables_different_methods'].keys()

dict_keys(['Cement_global_warming_potential_(gwp100)', 'Cement_cumulative_energy_demand_-_non-renewable_energy_resources'])

In [150]:
sector_activities=main_dict['Electricity']['activities']

lca_scores=compare_activities_multiple_methods(
        activities_list=sector_activities,
        methods=method_dict,
        identifier='Electricity',
        mode='absolute'
    )

lca_scores=small_inputs_to_other_column(lca_scores, cutoff=0.02)

scores_dict['Electricity']['lca_tables_different_methods']=lca_scores

Omitting activity name common prefix: 'electricity production, at biomass-fired IGCC power '
Omitting activity name common prefix: 'electricity production, at biomass-fired IGCC power '


In [152]:
scores_dict['Electricity']['lca_tables_different_methods'].keys()

dict_keys(['Electricity_global_warming_potential_(gwp100)', 'Electricity_cumulative_energy_demand_-_non-renewable_energy_resources'])

In [157]:
def sector_lca_scores(main_dict, method_dict):
    '''
    Generates the LCA scores for the sectors activities in the main dictionary 
    for the different methods in the method dictionary.

    It returns the main dictionary updated as scores dictionary which also holds the former information for each sector.
    The LCA scores are stored by method name in the respective sector dictionary within the main dictionary.
    '''

    # Initialize scores_dict as a copy of main_dict
    scores_dict = main_dict.copy()

    # Loop through each sector in main_dict
    for sector in main_dict.keys():
        # Extract activities for the current sector
        sector_activities = main_dict[sector]['activities']
        
        # Calculate LCA scores using the specified method
        lca_scores = compare_activities_multiple_methods(
            activities_list=sector_activities,
            methods=method_dict,
            identifier=sector,
            mode='absolute'
        )
        
        # Apply the small_inputs_to_other_column function with the cutoff value
        lca_scores = small_inputs_to_other_column(lca_scores, cutoff=0.02)
        
        # Save the LCA scores to the scores_dict
        scores_dict[sector]['lca_tables_different_methods'] = lca_scores

    return scores_dict

In [158]:
scores_dict=sector_lca_scores(sectors_dict, method_dict)

Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'electricity production, at biomass-fired IGCC power '
Omitting activity name common prefix: 'electricity production, at biomass-fired IGCC power '


In [156]:
scores_dict['Cement']['lca_tables_different_methods'].keys()

dict_keys(['Cement_global_warming_potential_(gwp100)', 'Cement_cumulative_energy_demand_-_non-renewable_energy_resources'])

In [185]:
# Function to add statistics to DataFrame
def add_statistics(df, column_name='total'):
    # Calculate mean, standard deviation, and IQR
    df['mean'] = df[column_name].mean()
    df['std'] = df[column_name].std()
    df['q1'] = df[column_name].quantile(0.25)
    df['q3'] = df[column_name].quantile(0.75)
    return df

In [196]:
def sector_lca_scores_to_excel(scores_dict, excel_file_name):
    # Prepare to save each LCA score table to a different worksheet in the same Excel file
    excel_file = excel_file_name
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        for sector in scores_dict.keys():
            lca_tables = scores_dict[sector]['lca_tables_different_methods']
            for method, table in lca_tables.items():
                # Create a DataFrame for the current LCA score table
                df = pd.DataFrame(table)
                 # Add statistics to the DataFrame
                df = add_statistics(df)
                # Add sector marker column
                df['sector']=str(sector)
                # Generate a worksheet name
                worksheet_name = f"{sector}_{method}"
                if len(worksheet_name) > 31:
                    worksheet_name = worksheet_name[:31]
                # Save the DataFrame to the Excel file in a new worksheet
                df.to_excel(writer, sheet_name=worksheet_name, index=False)

In [197]:
lca_scores_in_excel=sector_lca_scores_to_excel(scores_dict, 'output_v2.xlsx')

In [202]:
def sector_lca_scores_to_df(scores_dict):
    df_list=[]
    for sector in scores_dict.keys():
        lca_tables = scores_dict[sector]['lca_tables_different_methods']
        for method, table in lca_tables.items():
            # Create a DataFrame for the current LCA score table
            df = pd.DataFrame(table)
            # Add statistics to the DataFrame
            df = add_statistics(df)
            
            df_list.append(df)
    return df_list

In [204]:
df_list=sector_lca_scores_to_df(sectors_dict)


In [210]:
df=df_list[0]
df

,activity,product,location,unit,method,method unit,total,37430: Cement clinkers,33: Coke oven products; refined petroleum products; nuclear fuel,6511: Road transport services of freight,11010: Hard coal,other,mean,std,q1,q3
4,Portland,,PE,kilogram,global warming potential (GWP100),kg CO2-Eq,0.869433,0.824794,0.000000,0.000000,0.000000,0.043989,0.813069,0.04405,0.794423,0.837095
1,Portland,,CA-QC,kilogram,global warming potential (GWP100),kg CO2-Eq,0.841855,0.781589,0.000000,0.000000,0.000000,0.059457,0.813069,0.04405,0.794423,0.837095
0,Portland,,BR,kilogram,global warming potential (GWP100),kg CO2-Eq,0.832335,0.742414,0.043855,0.000000,0.000000,0.045301,0.813069,0.04405,0.794423,0.837095
5,Portland,,US,kilogram,global warming potential (GWP100),kg CO2-Eq,0.821278,0.759166,0.000000,0.000000,0.000000,0.061235,0.813069,0.04405,0.794423,0.837095
6,Portland,,ZA,kilogram,global warming potential (GWP100),kg CO2-Eq,0.814768,0.758633,0.000000,0.027084,0.000000,0.028688,0.813069,0.04405,0.794423,0.837095
3,Portland,,IN,kilogram,global warming potential (GWP100),kg CO2-Eq,0.774079,0.718396,0.000000,0.000000,0.016089,0.039142,0.813069,0.04405,0.794423,0.837095
2,Portland,,CH,kilogram,global warming potential (GWP100),kg CO2-Eq,0.737734,0.701263,0.000000,0.000000,0.000000,0.036028,0.813069,0.04405,0.794423,0.837095


In [208]:
import pandas as pd
import openpyxl
from openpyxl.chart import (
    ScatterChart,
    Reference,
    Series
)

# Helper function to convert a pandas DataFrame to a list of rows
def dataframe_to_rows(df, include_index=True, include_column_header=True):
    rows = []
    if include_column_header:
        rows.append([str(c) for c in df.columns])
    for _, row in df.iterrows():
        if include_index:
            rows.append([str(row.name)] + [str(v) for v in row])
        else:
            rows.append([str(v) for v in row])
    return rows

# Assuming you have a dictionary 'my_dict' and a dataframe 'df' in the dictionary
df = df_list[0]

# Create a new workbook
wb = openpyxl.Workbook()

# Write the dataframe to the first worksheet
ws_data = wb.active
for r in dataframe_to_rows(df, index=False, header=True):
    ws_data.append(r)

# Create a new worksheet for the chart
ws_chart = wb.create_sheet("Scatter Plot")

# Create the scatter chart
scatter_chart = ScatterChart()
scatter_chart.title = "Scatter Plot of Total Values"
scatter_chart.style = 13
scatter_chart.x_axis.title = "Index"
scatter_chart.y_axis.title = "Total"

# Get the data range for the scatter plot
x_values = Reference(ws_data, min_col=1, min_row=2, max_row=len(df)+1)
y_values = Reference(ws_data, min_col=df.columns.get_loc('total')+1, min_row=2, max_row=len(df)+1)

# Add the data series to the chart
scatter_chart.series.append(Series(y_values, x_values))

# Add the chart to the chart worksheet
ws_chart.add_chart(scatter_chart, "A1")

# Save the workbook
wb.save("scatter_plot.xlsx")

TypeError: dataframe_to_rows() got an unexpected keyword argument 'index'

PLOTS IN EXCEL

In [ ]:
#for each sector create a worksheet which holds scatter plots where the x axis shows the index and the y axis the total score

In [195]:
import openpyxl
from openpyxl.chart import ScatterChart, Reference, Series

# Load the Excel file
wb = openpyxl.load_workbook('output_v2.xlsx')

# Create a dictionary to store the charts for each sector
sector_charts = {}

# Loop through each worksheet
for worksheet_name in wb.sheetnames:
    # Extract the sector name from the worksheet name
    sector_name = worksheet_name.split('_')[0]

    # Get the worksheet
    ws = wb[worksheet_name]

    # Determine the number of rows in the worksheet
    row_count = ws.max_row

    # Find the column index for the 'total' column (assuming this is the column you want to plot)
    for col_num in range(1, ws.max_column + 1):
        if ws.cell(row=1, column=col_num).value == 'total':
            y_col = col_num
            break
    else:
        # If the 'total' column is not found, skip this worksheet
        continue

    # Create the scatter chart
    chart = ScatterChart()
    chart.title = f"LCA Scores - {worksheet_name}"
    chart.y_axis.title = str(ws['B1'].value) # Assuming the method is in cell B1
    chart.x_axis.title = 'activities indexes'

    # Define the data range for the chart
    y_values = Reference(ws, min_col=y_col, min_row=2, max_row=row_count) # Assuming the data starts in column 2
    x_values = Reference(ws, min_col=1, min_row=2, max_row=row_count)

    # Create a series and add it to the chart
    series = Series(y_values, x_values, title_from_data=False)
    chart.series.append(series)

    # Customize the series to show only markers (dots)
    series.marker.symbol = "circle"
    series.marker.size = 5
    series.graphicalProperties.line.noFill = True

    # Add the chart to the sector-specific worksheet
    if sector_name not in sector_charts:
        sector_charts[sector_name] = wb.create_sheet(f"Plot - {sector_name}")
    sector_charts[sector_name].add_chart(chart, f"A{len(sector_charts[sector_name]['A'])+1}")

# Save the updated workbook
wb.save('output_v2.xlsx')

In [198]:
# Load the Excel file
wb = openpyxl.load_workbook('output_v2.xlsx')

# Create a dictionary to store the charts for each sector
sector_charts = {}

# Iterate through the worksheets in the workbook
for ws in wb.worksheets:
    # Find the column with the 'method' label
    for col in range(1, ws.max_column + 1):
        if str(ws.cell(row=1, column=col).value) == 'method':
            sector_name = str(ws.cell(row=2, column=col).value)
            break
    else:
        # If 'method' column not found, skip this worksheet
        continue

    # Check if the sector is already in the dictionary
    if sector_name not in sector_charts:
        # Create a new worksheet for the sector
        sector_ws = wb.create_sheet(sector_name)
        sector_charts[sector_name] = sector_ws

    # Get the number of rows in the worksheet
    row_count = ws.max_row

    # Create the scatter chart
    chart = ScatterChart()
    chart.title = f"LCA Scores - {sector_name}"
    chart.y_axis.title = sector_name
    chart.x_axis.title = 'activities indexes'

    # Define the data range for the chart
    y_col = next(col for col in range(1, ws.max_column + 1) if str(ws.cell(row=1, column=col).value) != 'method')  # Assuming the data starts in the column after 'method'
    y_values = Reference(ws, min_col=y_col, min_row=2, max_row=row_count)
    x_values = Reference(ws, min_col=1, min_row=2, max_row=row_count)

    # Create a series and add it to the chart
    series = Series(y_values, x_values, title_from_data=False)
    chart.series.append(series)

    # Customize the series to show only markers (dots)
    series.marker.symbol = "circle"
    series.marker.size = 5
    series.graphicalProperties.line.noFill = True

    # Add the chart to the sector worksheet
    sector_charts[sector_name].add_chart(chart, "A1")

# Save the workbook
wb.save('output_v2.xlsx')

c:\Users\fried\miniconda3\envs\premise\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.chart import ScatterChart, Reference, Series

# Assuming you have a DataFrame called 'df' with a column named 'total'
# If not, replace 'df' with your actual DataFrame name

# Step 1: Save the DataFrame to an Excel file
excel_file = 'output_v1.xlsx'
df.to_excel(excel_file, index=False)

# Step 2: Use openpyxl to create the scatter chart
wb = load_workbook(excel_file)
ws = wb.active

# Determine the number of rows in the worksheet
row_count = ws.max_row

# Create the scatter chart
chart = ScatterChart()
chart.title = "LCA Scores"
chart.y_axis.title = str(df['method'])
chart.x_axis.title = 'activities indexes'

# Define the data range for the chart
y_values = Reference(ws, min_col=df.columns.get_loc('total') + 1, min_row=2, max_row=row_count) #add sorting step?
x_values = Reference(ws, min_col=1, min_row=2, max_row=row_count)

# Create a series and add it to the chart
series = Series(y_values, x_values, title_from_data=False)
chart.series.append(series)

# Customize the series to show only markers (dots)
series.marker.symbol = "circle"
series.marker.size = 5
series.graphicalProperties.line.noFill = True

# Add the chart to the worksheet
ws.add_chart(chart, "H2")

# Save the workbook
wb.save(excel_file)

print(f"Scatter chart with dots has been created and saved in {excel_file}")

Scatter chart with dots has been created and saved in data.xlsx


In [ ]:
import openpyxl
import openpyxl.chart

wb = openpyxl.Workbook()
ws = wb.active

data=lca_scores_agg

# Write dictionary keys to the first row (header)
for col_num, key in enumerate(data.keys(), start=1):
    ws.cell(row=1, column=col_num, value=key)

# Write dictionary values to subsequent rows
for row_num, row_values in enumerate(zip(*data.values()), start=2):
    for col_num, value in enumerate(row_values, start=1):
        ws.cell(row=row_num, column=col_num, value=value)

wb.save('test_wb.xlsx')
#openpyxl.chart.scatter_chart(scatterStyle='marker')


In [ ]:
# Run bw 2 analyzer for the 5 sectors to then get stacked bar plots for several methods. Save each method in a different sheet.

In [ ]:
# Get ecoinvent and filter it also for the sectors. Plot level 3. Different methods in one sheet